<a href="https://colab.research.google.com/github/12003586/Hackaton/blob/main/Hackaton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 4.8 MB/s 
     |████████████████████████████████| 15.7 MB 223 kB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.3
    Uninstalling numpy-1.19.3:
      Successfully uninstalled numpy-1.19.3
  Attempting uninstall: keras
    Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
plotnine 0.8.0 requires scipy>=1.5.0, but you have scipy 1.4.1 which is incompatible.
jaxlib 0.3.25+cuda11.cudnn805 requires scipy>=1.5, but you have scipy 1.4.1 which is incompatible.
jax 0.3.25 requires scipy>=1.5, but you have scipy 1.4.1 which is incompatible.
imageai 2.1.6 requires keras==2.4.3, but you ha

In [2]:
!pip install imageai 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import random

from os import listdir
from os.path import isfile, join

import urllib.request
import requests
from io import BytesIO

from PIL import Image
from PIL.ImageEnhance import Color, Contrast, Brightness

#from tensorflow import keras
from tensorflow.keras.models import Model as tf_Model
from tensorflow.keras.layers import Flatten, Dense, Conv2D, Dropout, MaxPooling2D
from tensorflow.keras import optimizers as tf_optim

from sklearn.ensemble import AdaBoostRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer, OneHotEncoder

import sklearn.metrics as metrics

from imageai.Classification.Custom import ClassificationModelTrainer

from google.colab import drive
drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
%cd /content/drive/My Drive/Colab Notebooks/Kaggle_Fruits360.zip/

[Errno 20] Not a directory: '/content/drive/My Drive/Colab Notebooks/Kaggle_Fruits360.zip/'
/content


In [16]:
git_url = "https://raw.githubusercontent.com/marcusklasson/GroceryStoreDataset/master/dataset"
git_sub_folder = "/train/Fruit/Apple/Golden-Delicious/"
#image_name = 'Golden-Delicious_001.jpg'
response = requests.get(git_url + git_sub_folder )
image = Image.open(BytesIO(response.content))

UnidentifiedImageError: ignored

In [33]:
from tensorflow.keras.applications.vgg16 import VGG16

image_shape = (224, 224, 3)

base_model = VGG16(input_shape = image_shape,
    include_top = False,
    weights = 'imagenet')

base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [34]:
n_outputs = 10

x = Conv2D(filters = 1024, kernel_size=(2,2), padding='valid')(base_model.output)
x = Conv2D(filters = 1024, kernel_size=(2,2), padding='same')(x)
x = MaxPooling2D((2,2))(x)
x = Flatten()(x)
x = Dropout(rate=0.25)(x)
x = Dense(128, activation='relu')(x)
x = Dense(n_outputs, activation='sigmoid')(x)

model = tf_Model(base_model.input, x, name = 'expanded_model')

model.summary()

Model: "expanded_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)    

In [35]:
model.compile(optimizer = tf_optim.RMSprop(learning_rate=0.0001), loss = "mean_scuared_error", metrics = ["mean_scuared_error"])


In [36]:
train_it = datagen.flow_from_directory('Kaggle_Fruits360.zip', 
                                       target_size=(224, 224), 
                                       color_mode='rgb', 
                                       class_mode="categorical")
# load and iterate validation dataset
valid_it = datagen.flow_from_directory('Kaggle_Fruits360.zip', 
                                      target_size=(224,224), 
                                      color_mode='rgb', 
                                      class_mode="categorical")

NotADirectoryError: ignored